In [15]:
import pandas as pd
import numpy as np

In [16]:
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
import re
from pandas.testing import assert_series_equal, assert_frame_equal

In [60]:
pd.options.display.max_rows = 999

## Comparing files

In [345]:
proc_new = pd.read_csv('data/processed/new.csv')
proc_old = pd.read_csv('data/processed/old.csv')
anom_new = pd.read_csv('data/anomalies/new.csv')
anom_old = pd.read_csv('data/anomalies/old.csv')

In [316]:
cleaned = pd.read_csv('data/cleaned.csv', dtype='object')

In [346]:
proc_new = proc_new.sort_values(by=['pai_account_id', 'date'])
proc_old = proc_old.sort_values(by=['pai_account_id', 'date'])

In [347]:
common_accounts = list(set(proc_old.pai_account_id).intersection(set(proc_new.pai_account_id)))

In [348]:
proc_old = proc_old[proc_old.pai_account_id.isin(common_accounts)] 
proc_new = proc_new[proc_new.pai_account_id.isin(common_accounts)]

In [349]:
proc_old.shape, proc_new.shape

((501107, 7), (501713, 7))

In [351]:
good, bad = 0,0
accounts_to_fix = []
for acc in common_accounts:
    old = proc_old[proc_old.pai_account_id == acc].reset_index(drop=True)
    new = proc_new[proc_new.pai_account_id == acc].reset_index(drop=True)
    shared_dates = set(old.date).intersection(new.date)
    set_diff = list((set(old.date) - set(new.date)).union(set(new.date) - set(old.date)))
    if len(shared_dates) > 0:
        print(len(shared_dates), len(set_diff))
        new = new[new.date.isin(shared_dates)].reset_index(drop=True)
        old = old[old.date.isin(shared_dates)].reset_index(drop=True)
    try:
        assert_frame_equal(old, new)
        good += 1
    except AssertionError:
        bad += 1
        accounts_to_fix.append(acc)

389 0
551 2
700 0
365 0
244 0
93 0
1362 2
123 0
698 2
682 0
384 0
1364 2
342 0
55 2
343 0
196 2
371 0
103 0
88 0
421 2
92 0
253 0
389 0
267 2
1008 2
1031 2
484 0
422 2
91 0
106 0
378 0
822 0
92 0
414 0
39 0
92 0
339 0
252 0
363 0
12 0
12 0
167 0
452 2
483 2
1345 2
151 0
1186 0
447 0
92 0
160 0
1042 2
676 2
204 0
262 0
202 0
92 0
366 2
118 0
634 2
318 2
212 0
126 0
1363 2
733 0
474 0
305 0
93 0
490 0
365 0
61 0
941 0
1117 0
65 0
1105 0
91 0
91 0
516 0
974 2
142 0
376 0
363 0
1378 2
331 2
369 0
91 0
851 0
583 0
1061 2
363 0
98 0
123 0
355 0
91 0
377 0
1067 0
515 0
1391 2
1032 0
992 2
469 0
90 0
230 2
9 0
812 2
84 0
373 0
411 0
1034 2
290 0
459 0
42 0
41 0
1085 0
281 0
606 2
422 0
259 0
612 0
273 0
950 2
325 0
331 0
33 0
379 0
1148 2
1220 0
189 0
93 0
88 0
928 0
401 0
647 2
84 0
1361 2
614 0
558 2
699 0
674 0
72 0
95 0
272 2
907 2
613 2
232 0
332 0
584 0
395 0
23 0
358 0
211 0
103 0
1390 2
599 0
395 0
203 2
81 2
187 2
408 0
92 0
701 0
894 2
693 2
21 0
307 2
377 0
106 0
59 0
1034 0
1093 0


In [352]:
len(accounts_to_fix)

0

In [353]:
good, bad = 0,0
accounts_to_fix = []
for acc in common_accounts:
    old = proc_old[proc_old.pai_account_id == acc].reset_index(drop=True)
    new = proc_new[proc_new.pai_account_id == acc].reset_index(drop=True)
    shared_dates = set(old.date).intersection(new.date)
    set_diff = list((set(old.date) - set(new.date)).union(set(new.date) - set(old.date)))
    if len(shared_dates) > 0:
        new = new[new.date.isin(shared_dates)].reset_index(drop=True)
        old = old[old.date.isin(shared_dates)].reset_index(drop=True)
    try:
        assert_frame_equal(old, new)
        good += 1
    except AssertionError as e:
        bad += 1
        accounts_to_fix.append(acc)
        print(acc)
        print(e)

In [339]:
len(accounts_to_fix)

0

In [354]:
extra_dates = {}
for acc in common_accounts:
    old = proc_old[proc_old.pai_account_id == acc]
    new = proc_new[proc_new.pai_account_id == acc]
    set_diff = list((set(old.date) - set(new.date)).union(set(new.date) - set(old.date)))
    if len(set_diff) > 0:
        extra_dates[acc] = set_diff

In [355]:
extra_dates['001b000003g8u3B']

['2019-02-26', '2019-02-25']

In [356]:
{k:len(v) for k,v in extra_dates.items()}

{'001b000003wxIch': 2,
 '001b00000068PkN': 2,
 '001b000003j4pGR': 2,
 '001b000000aQadY': 2,
 '001b000003iTU3m': 2,
 '001b000000c3fej': 2,
 '0010J00001oqsqw': 2,
 '0010J00001sqGlJ': 2,
 '001b000003LhWev': 2,
 '001b000000bQPWF': 2,
 '001b000003J0tbY': 2,
 '001b000003jz9C7': 2,
 '0010J00001oc4O9': 2,
 '001b000000l8Y79': 2,
 '001b000003LhZDa': 2,
 '001b000003jzdhk': 2,
 '001b000003UxdBb': 2,
 '001b000003mXdDB': 2,
 '001b000003ydcqU': 2,
 '001b000000XoQOE': 2,
 '001b000003OGF51': 2,
 '001b000000I5WIl': 2,
 '0010J00001pFkEw': 2,
 '001b000001UmhjW': 2,
 '001b000000aS1kM': 2,
 '001b000003OFENz': 2,
 '0010J00001or2yM': 2,
 '001b000003ZuU1c': 2,
 '001b000003LfS6f': 2,
 '001b000003oyztN': 2,
 '001b000003OFcXW': 2,
 '001b000001EiCdH': 2,
 '001b000003nfVdE': 2,
 '001b000000aSU9O': 2,
 '001b000001pmytA': 2,
 '0010J00001pFtOl': 2,
 '001b000003TPm7H': 2,
 '001b0000019w95b': 2,
 '001b0000001yGLf': 2,
 '0010J00001sr81X': 2,
 '0010J00001yTJxk': 2,
 '0010J00001yQjwS': 2,
 '001b000003Uwe4a': 2,
 '001b00000

In [357]:
i = '001b000002LQE31'
old = proc_old[proc_old.pai_account_id == i].reset_index(drop=True)
new = proc_new[proc_new.pai_account_id == i].reset_index(drop=True)
shared_dates = set(old.date).intersection(new.date)
if len(shared_dates) > 0:
    new = new[new.date.isin(shared_dates)].reset_index(drop=True)
    old = old[old.date.isin(shared_dates)].reset_index(drop=True)
try:
    assert_frame_equal(old, new)
except AssertionError as e:
    print(e)

In [312]:
inter_ind = proc_old.index.intersection(proc_new.index)

In [313]:
test_new = proc_new.loc[inter_ind]
test_old = proc_old.loc[inter_ind]

In [314]:
test_new.shape, test_old.shape

((490164, 7), (490164, 7))

In [648]:
joined = pd.concat([test_old, test_new], axis=1)

Churn flag:

In [651]:
joined[joined.iloc[:,0] != joined.iloc[:,5]]

churn_flag  client_id     profile stage  \
pai_account_id  date                                                  
001b000002LQE31 2016-02-12           1          1  enterprise   poc   
                2016-03-14           0          1  enterprise   poc   

                           stage_start_date  churn_flag  client_id  \
pai_account_id  date                                                 
001b000002LQE31 2016-02-12       2016-01-13           0          1   
                2016-03-14       2016-01-13           1          1   

                               profile stage stage_start_date  
pai_account_id  date                                           
001b000002LQE31 2016-02-12  enterprise   poc       2016-01-13  
                2016-03-14  enterprise   poc       2016-01-13

It's a bad account - was poc all its life. In new it's churned on last day, in old it churns in middle then goes back to 0!

In [297]:
proc_new.columns.get_loc('date')

2

In [296]:
proc_new[proc_new.churn_flag == 1]

,churn_flag,client_id,date,pai_account_id,profile,stage,stage_start_date
978,1,1,2018-10-27,0010J00001oc5JA,smb,paying,2018-01-22
1608,1,1,2018-08-25,0010J00001oc8CR,enterprise,poc,2018-04-30
2503,1,1,2018-10-01,0010J00001oc8VO,smb,paying,2018-01-02
3664,1,1,2018-11-27,0010J00001ooUlg,smb,paying,2018-05-28
4510,1,1,2018-09-23,0010J00001ooVZB,smb,paying,2018-01-19
5131,1,1,2018-03-26,0010J00001opOlf,enterprise,poc,2017-10-24
6307,1,1,2018-07-08,0010J00001opXUT,enterprise,poc,2017-11-16
7542,1,1,2018-11-30,0010J00001oq1Ks,smb,paying,2018-06-03
7725,1,1,2018-06-04,0010J00001oq3f9,enterprise,poc,2017-12-04
7992,1,1,2018-05-03,0010J00001oqadE,enterprise,poc,2018-01-11


In [294]:
proc_new.pai_account_id.unique()[0]

'0010J00001oc4O9'

In [662]:
joined.loc['001b000002LQE31'].head(90)

,churn_flag,client_id,profile,stage,stage_start_date,churn_flag,client_id,profile,stage,stage_start_date
date,,,,,,,,,,
2016-01-13,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-14,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-15,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-16,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-17,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-18,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-19,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-20,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13
2016-01-21,0,1,enterprise,poc,2016-01-13,0,1,enterprise,poc,2016-01-13


In [666]:
cleaned[cleaned.pai_account_id == '001b000002LQE31'].churn_in_stage.value_counts()

1    95
Name: churn_in_stage, dtype: int64

Client_id, profile(!):

In [652]:
joined[joined.iloc[:,1] != joined.iloc[:,6]]

,,churn_flag,client_id,profile,stage,stage_start_date,churn_flag,client_id,profile,stage,stage_start_date
pai_account_id,date,,,,,,,,,,


In [653]:
joined[joined.iloc[:,2] != joined.iloc[:,7]]

,,churn_flag,client_id,profile,stage,stage_start_date,churn_flag,client_id,profile,stage,stage_start_date
pai_account_id,date,,,,,,,,,,


Stage (1503):

In [667]:
temp = joined[joined.iloc[:,3] != joined.iloc[:,8]]
temp

churn_flag  client_id     profile   stage  \
pai_account_id  date                                                    
001b000000c5VoN 2017-03-19           0          1  enterprise  paying   
001b000000p20V8 2017-04-16           0          1         smb  paying   
                2017-04-17           0          1         smb  paying   
                2017-04-18           0          1         smb  paying   
                2017-04-19           0          1         smb  paying   
                2017-04-20           0          1         smb  paying   
                2017-04-21           0          1         smb  paying   
                2017-04-22           0          1         smb  paying   
                2017-04-23           0          1         smb  paying   
                2017-04-24           0          1         smb  paying   
                2017-04-25           0          1         smb  paying   
                2017-04-26           0          1         smb  paying   
                2017-04-27           0          1         smb  paying   
                2017-04-28           0          1         smb  paying   
                2017-04-29           0          1         smb  paying   
                2017-04-30           0          1         smb  paying   
                2017-05-01           0          1         smb  paying   
                2017-05-02           0          1         smb  paying   
                2017-05-03           0          1         smb  paying   
                2017-05-04           0          1         smb  paying   
                2017-05-05           0          1         smb  paying   
                2017-05-06           0          1         smb  paying   
                2017-05-07           0          1         smb  paying   
                2017-05-08           0          1         smb  paying   
                2017-05-09           0          1         smb  paying   
                2017-05-10           0          1         smb  paying   
                2017-05-11           0          1         smb  paying   
                2017-05-12           0          1         smb  paying   
                2017-05-13           0          1         smb  paying   
                2017-05-14           0          1         smb  paying   
                2017-05-15           0          1         smb  paying   
                2017-05-16           0          1         smb  paying   
                2017-05-17           0          1         smb  paying   
                2017-05-18           0          1         smb  paying   
                2017-05-19           0          1         smb  paying   
                2017-05-20           0          1         smb  paying   
                2017-05-21           0          1         smb  paying   
                2017-05-22           0          1         smb  paying   
                2017-05-23           0          1         smb  paying   
                2017-05-24           0          1         smb  paying   
                2017-05-25           0          1         smb  paying   
                2017-05-26           0          1         smb  paying   
                2017-05-27           0          1         smb  paying   
                2017-05-28           0          1         smb  paying   
                2017-05-29           0          1         smb  paying   
                2017-05-30           0          1         smb  paying   
                2017-05-31           0          1         smb  paying   
                2017-06-01           0          1         smb  paying   
                2017-06-02           0          1         smb  paying   
                2017-06-03           0          1         smb  paying   
                2017-06-04           0          1         smb  paying   
                2017-06-05           0          1         smb  paying   
                2017-06-06           0          1         smb  paying   
             

All changes are from paying to poc

In [668]:
print(temp.iloc[:,3].value_counts(), temp.iloc[:,8].value_counts())

paying    1503
Name: stage, dtype: int64 poc    1503
Name: stage, dtype: int64


Stage start date (11510!):

This is understandable since if we changed the stage, we change the stage start date all along

In [655]:
joined[joined.iloc[:,4] != joined.iloc[:,9]]

churn_flag  client_id     profile   stage  \
pai_account_id  date                                                    
001b000000c5VoN 2017-03-19           0          1  enterprise  paying   
                2017-03-20           0          1  enterprise  paying   
                2017-03-21           0          1  enterprise  paying   
                2017-03-22           0          1  enterprise  paying   
                2017-03-23           0          1  enterprise  paying   
                2017-03-24           0          1  enterprise  paying   
                2017-03-25           0          1  enterprise  paying   
                2017-03-26           0          1  enterprise  paying   
                2017-03-27           0          1  enterprise  paying   
                2017-03-28           0          1  enterprise  paying   
                2017-03-29           0          1  enterprise  paying   
                2017-03-30           0          1  enterprise  paying   
                2017-03-31           0          1  enterprise  paying   
                2017-04-01           0          1  enterprise  paying   
                2017-04-02           0          1  enterprise  paying   
                2017-04-03           0          1  enterprise  paying   
                2017-04-04           0          1  enterprise  paying   
                2017-04-05           0          1  enterprise  paying   
                2017-04-06           0          1  enterprise  paying   
                2017-04-07           0          1  enterprise  paying   
                2017-04-08           0          1  enterprise  paying   
                2017-04-09           0          1  enterprise  paying   
                2017-04-10           0          1  enterprise  paying   
                2017-04-11           0          1  enterprise  paying   
                2017-04-12           0          1  enterprise  paying   
                2017-04-13           0          1  enterprise  paying   
                2017-04-14           0          1  enterprise  paying   
                2017-04-15           0          1  enterprise  paying   
                2017-04-16           0          1  enterprise  paying   
                2017-04-17           0          1  enterprise  paying   
...                                ...        ...         ...     ...   
001b000003s29Mp 2018-07-14           0          1  enterprise  paying   
                2018-07-15           0          1  enterprise  paying   
                2018-07-16           0          1  enterprise  paying   
                2018-07-17           0          1  enterprise  paying   
                2018-07-18           0          1  enterprise  paying   
                2018-07-19           0          1  enterprise  paying   
                2018-07-20           0          1  enterprise  paying   
                2018-07-21           0          1  enterprise  paying   
                2018-07-22           0          1  enterprise  paying   
                2018-07-23           0          1  enterprise  paying   
                2018-07-24           0          1  enterprise  paying   
                2018-07-25           0          1  enterprise  paying   
                2018-07-26           0          1  enterprise  paying   
                2018-07-27           0          1  enterprise  paying   
                2018-07-28           0          1  enterprise  paying   
                2018-07-29           0          1  enterprise  paying   
                2018-07-30           0          1  enterprise  paying   
                2018-07-31           0          1  enterprise  paying   
                2018-08-01           0          1  enterprise  paying   
                2018-08-02           0          1  enterprise  paying   
                2018-08-03           0          1  enterprise  paying   
                2018-08-04           0          1  enterprise  paying   
             

In [643]:
cols = list(test_new)
for i in range(len(cols)):
    print(cols[i])
    assert_series_equal(test_old.iloc[:,i], test_new.iloc[:,i])

churn_flag


AssertionError: Series are different

Series values are different (0.00046 %)
[left]:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]
[right]: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]

Now let's look at the set diff:

In [669]:
diff_old_ind = proc_old.index.difference(proc_new.index)
len(diff_old_ind)

In [671]:
diff_new_ind = proc_new.index.difference(proc_old.index)
len(diff_new_ind)

16

In [672]:
diff_new_ind

MultiIndex(levels=[['001b000000p20V8', '001b000003hW9ZA', '001b000003j7RxW', '001b000003jzTcQ', '001b000003jzdhk', '001b000003kf21M'], ['2017-04-14', '2017-04-15', '2017-04-21', '2017-04-22', '2017-04-23', '2017-04-24']],
           labels=[[0, 1, 1, 1, 2, 2, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5], [1, 3, 4, 5, 4, 5, 0, 1, 2, 3, 4, 5, 2, 3, 4, 5]],
           names=['pai_account_id', 'date'],
           sortorder=0)

In [673]:
diff_old_ind

MultiIndex(levels=[['0010J00001oc4O9', '0010J00001oc4q3', '0010J00001oc5JA', '0010J00001oc8BY', '0010J00001oc8CR', '0010J00001oc8DF', '0010J00001oc8VO', '0010J00001ooU2z', '0010J00001ooUjf', '0010J00001ooUlg', '0010J00001ooV78', '0010J00001ooVZB', '0010J00001ooX0r', '0010J00001opSZk', '0010J00001opUHM', '0010J00001opZQT', '0010J00001opa7S', '0010J00001oq1Ks', '0010J00001oqE7n', '0010J00001oqgkS', '0010J00001oqpnf', '0010J00001oqsqw', '0010J00001or0lk', '0010J00001or2yM', '0010J00001orC6W', '0010J00001orC7j', '0010J00001os9YZ', '0010J00001osExa', '0010J00001pFQ4e', '0010J00001pFk2v', '0010J00001pFkEw', '0010J00001pFtOl', '0010J00001pG7pF', '0010J00001sOm4D', '0010J00001spJdB', '0010J00001spR4s', '0010J00001spmKU', '0010J00001spvzw', '0010J00001spwSF', '0010J00001sq6eQ', '0010J00001sqGlJ', '0010J00001sqXoG', '0010J00001sqnWX', '0010J00001sqsnY', '0010J00001sqx0M', '0010J00001sr81X', '0010J00001sriLY', '0010J00001xx0Zv', '0010J00001yQjwS', '0010J00001yRutG', '0010J00001yTJxk', '0010J00001